# Dimensionality Reduction

* Many Machine Learning problems involve thousands or even millions of features for each training instance. Not only does this make training extremely slow, it can also make it much harder to find a good solution, as we will see. This problem is often referred to as the curse of dimensionality.

* Reducing dimensionality does lose some imformation, it may also make our system perform slightly worse. we should first try to train our system with the original data before considering using dimensionality reduction if training is too slow.

* Reducing the dimensionality of training data may filter out some noise and unnecessary details and thus result in higher performance(but in general it won't)

## The Curse of Dimensionality

* High-dimensional datasets are at risk of being very sparse: most training instances are likely to be far away from each other.

* A new instance will likely be far away from any training instance, making predictions much less reliable than in lower dimensions.

* The more dimensions the training set has, the greater the risk of overfitting it.

## Main Approaches for Dimensionality Reduction

### Projection

* In most real-world problems, training instances are not spread out uniformly across all dimensions. all training instances actually lie within (or close to) a much lower-dimensional subspace of the high-dimensional space.

* Projection is not always the best approach to dimensionality reduction. In many case the subspace may twist and turn.

### Manifold Learning

* a d-D manifold is a d-D shape that can be bent and twisted in a higher-dimensional space.

* Manifold Learning relies on the manifold assumption, also called the manifold hypothesis, which holds that most real-world high-dimensional datasets lie close to a much lower-dimensional manifold.

* The manifold assumption is often accompanied by another implicit assumption: that the task at hand(e.g., classification or regression) will be simpler if expressed in the lower-dimensional space of the manifold. (this assumption does not always hold)

* If we reduce the dimensionality of our training set before training a model, it will definitely speed up training, but it may not always lead to a better or simpler solution.

## PCA (Principal Component Analysis)

* The PCA identifies the hyperplane that lies closest to the data, and then it projects the data onto it.

### Preserving the Variance

* Select the axis that preserves the maximum amount of variance will most likely lose less information than the other projections.

* Another way to justify this choice is that it is the axis that minimizes the mean squared distance between the original dataset and its projection onto that axis.

### Principal Components

* The unit vector that defines the ith axis is called the ith principal component(PC).

* We can use Singular Value Decomposition to find the principal components of a training set.

### Projecting Down to d Dimensions

* Once we have identified all the principal components, we can reduce the dimensionality of the dataset down to d dimensions by projecting it onto the hyperplane defined by the first d principal components.

### Using sklearn


In [3]:
from sklearn.decomposition import PCA
import numpy as np

# build 3D dataset
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)


pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

### Explained Variance Ratio

* Explained Variance Ratio indicated the proportion of the dataset's variance that lies along the axis of each principal component.

In [4]:
print(pca.explained_variance_ratio_)

[ 0.84248607  0.14631839]


### Choosing the Right Number of Dimensions

* It is generally preferable to choose the number of dimensions that add up to a sufficiently large portion of the variance(e.g., 95%).

In [6]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

In [7]:
d

2

In [8]:
# the same but better way

pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

In [9]:
X_reduced

array([[ 1.26203346,  0.42067648],
       [-0.08001485, -0.35272239],
       [ 1.17545763,  0.36085729],
       [ 0.89305601, -0.30862856],
       [ 0.73016287, -0.25404049],
       [-1.10436914,  0.20204953],
       [ 1.27265808,  0.46781247],
       [-0.44933007,  0.67736663],
       [-1.09356195, -0.04467792],
       [-0.66177325, -0.28651264],
       [ 1.04466138, -0.11244353],
       [-1.05932502,  0.31189109],
       [ 1.13761426,  0.14576655],
       [ 1.16044117,  0.36481599],
       [-1.00167625,  0.39422008],
       [ 0.2750406 , -0.34391089],
       [-0.45624787,  0.69707573],
       [-0.79706574, -0.26870969],
       [-0.66924929,  0.65520024],
       [ 1.30679728,  0.37671343],
       [-0.6626586 , -0.32706423],
       [ 1.25387588,  0.56043928],
       [ 1.04046987, -0.08727672],
       [ 1.26047729,  0.1571074 ],
       [-1.09786649,  0.38643428],
       [-0.7130973 ,  0.64941523],
       [ 0.17786909, -0.43609071],
       [-1.02975735,  0.33747452],
       [ 0.94552283,

* Another option is to plot the explained variance as a function of the number of dimensions. There will usually be an elbow in the curve, where the explained variance stops growing fast. we can think of this as the intrinsic dimensionality of the dataset. 

### PCA for Compression

* Applying PCA to the MNIST dataset while preserving 95% of its variance. we should find that each instance will have just over 150 features, instead of the original 784 features. So while most of the variance is preserved, the dataset is now less than 20% of its orginal size.

* It is also possible to decompress the reduced dataset by applying the inverse transformation of the PCA projection. Of course this won't give we back the original data, since the projection lost a bit of information.

* The mean squared distance between the original data and the reconstructed data is called the reconstrcution error.

### Incremental PCA

* One problem with the preceding implementation of PCA is that it requires the whole training set to fit in memory in order for the SVD algorithm to run.

* Incremental PCA(IPCA) algorithms have been developed: we can split the training set into mini-batches and feed an IPCA algorithm one mini-batch at a time. 

### Randomized PCA

* Sklearn offers another option to perform PCA, called Randomized PCA. This is a stochastic algorithm that quickly finds an approximation of the first d principal components. Its computaional complexity is O(m×d^2) + O(d^3), instead of O(m×n^2) + O(n^3), so it is dramatically faster than the previous algorithms when d is much smaller than n.

### Kernal PCA

* use kernal trick in PCA method.

## LLE (Locally Linear Embedding)

* LLE is a very powerful nonlinear dimensionality reduction(NLDR).
